In [1]:
#process the pytorch fitting outfit into single large data files with calcs for plotting

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from __future__ import print_function
import numpy as np
#from tqdm import tqdm
import matplotlib.colors as colors
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
from matplotlib import cm
plt.rcParams.update({'font.size': 14, 'lines.markersize' : 4})
from scipy.interpolate import interp1d
from scipy.optimize import curve_fit
import sys, os
import time, datetime
from datetime import timedelta
import pandas as pd
sys.path.append('/reg/neh/home/tdd14/modules/cart2pol/cart2pol')
import cart2pol
from cart2pol import PolarRebin
from torchrecon_funcs_clean import *

In [3]:
from matplotlib import cm
from matplotlib.colors import ListedColormap
my_cmap = cm.get_cmap('jet')
xr = np.linspace(0, 1, 2000)
my_cmap = my_cmap(xr)
my_cmap[0:2,-1] = 0
my_cmap = ListedColormap(my_cmap)

In [4]:
#where the vNbases are
basesdir = '/reg/d/psdm/tmo/tmox51020/results/paris/circularpol_vNbases_dipole_Feb/'
PR = PolarRebin('/reg/d/psdm/tmo/tmox51020/results/paris/recon/PR_c32_r32_th32.h5') 


In [20]:
#where the recon output files are saved
basedir = '/reg/d/psdm/tmo/tmox51020/results/paris/recon/torch_test_clean'

files = np.array(os.listdir(basedir))
files = files[np.array(['.npy' in file for file in files])]
#files = sorted(files, key=lambda f: float(f.split('run')[0]))
print(len(files))
files

180


array(['pytorchrecon_5_Up0.3_seed0.npy', 'pytorchrecon_0_Up1.0_seed1.npy',
       'pytorchrecon_8_Up0.4_seed1.npy', 'pytorchrecon_1_Up0.2_seed2.npy',
       'pytorchrecon_2_Up1.0_seed0.npy', 'pytorchrecon_7_Up0.3_seed1.npy',
       'pytorchrecon_9_Up0.1_seed1.npy', 'pytorchrecon_5_Up0.2_seed0.npy',
       'pytorchrecon_6_Up1.0_seed2.npy', 'pytorchrecon_7_Up0.2_seed1.npy',
       'pytorchrecon_8_Up0.5_seed1.npy', 'pytorchrecon_1_Up0.3_seed2.npy',
       'pytorchrecon_5_Up0.1_seed0.npy', 'pytorchrecon_4_Up0.4_seed0.npy',
       'pytorchrecon_6_Up0.4_seed1.npy', 'pytorchrecon_9_Up0.3_seed1.npy',
       'pytorchrecon_0_Up0.5_seed2.npy', 'pytorchrecon_7_Up0.1_seed1.npy',
       'pytorchrecon_4_Up0.5_seed0.npy', 'pytorchrecon_8_Up1.0_seed2.npy',
       'pytorchrecon_9_Up0.2_seed1.npy', 'pytorchrecon_0_Up0.4_seed2.npy',
       'pytorchrecon_6_Up0.5_seed1.npy', 'pytorchrecon_1_Up0.1_seed2.npy',
       'pytorchrecon_0_Up0.3_seed2.npy', 'pytorchrecon_9_Up0.5_seed1.npy',
       'pytorchrecon_6_Up

In [6]:
l = 13
dat = np.ndarray(len(files), dtype = [('seed', int),('ID', int),('Up', float), ('Qs', float, 2*36-1),\
                                      ('costs', float,l), ('costM', float,l), ('costW', float,l), ('costS', float,l),\
                                      ('costQ', float, l), ('eshift',float), ('M_wght', float), ('Spec_wght', float),\
                                      ('Smooth_wght', float),\
                                      ('costMn', float,l), ('costWn', float,l), ('costSn', float,l),\
                                           ])

In [7]:
for i,f in enumerate(files):
    test = np.load(os.path.join(basedir,f), allow_pickle=True).item()
    for n in dat.dtype.names[:]:
        try:
            dat[i][n]=test[n]
        except:
            if n == 'costMn':
                try:
                    dat[i][n]=np.array(test['costM'])/test['M_wght']
                except:
                    dat[i][n] = 0
            elif n == 'costWn':
                try:
                    dat[i][n]=np.array(test['costW'])/test['Spec_wght']
                except:
                    dat[i][n] = 0
            elif n == 'costSn':
                try:
                    dat[i][n]=np.array(test['costS'])/test['Smooth_wght']
                except:
                    dat[i][n] = 0 
                
            elif n == 'Etvar':
                continue
            else: print(n)

In [8]:
for i,j in enumerate(np.unique(dat['ID'])):
    print('ID %1.0f, # files %1.0f' %(j,len(dat[dat['ID'] == j])))

ID 0, # files 6
ID 1, # files 6
ID 2, # files 6
ID 3, # files 6
ID 4, # files 6
ID 5, # files 6
ID 6, # files 6
ID 7, # files 6
ID 8, # files 6
ID 9, # files 6


In [9]:
Ups = np.unique(dat['Up'])
Ups

array([0.4, 1. ])

In [10]:
mfn = test['mfn'] 
gfn = test['gfn'] 
gfsig = test['gfsig']
eshift = test['eshift']
print(mfn,gfn,gfsig, eshift)
runnum = test['runnum']
gfcart = 0.25 #applied to the simulated image to try to make it match the measured
mfcart = 1
mdatname = test['mdatname']
specdatname = test['specdatname']
thresh = test['thresh']
mask = test['mask']
maskhm = test['maskhm']
pixpereV = test['pixpereV']

25 15 0.985 0


In [16]:
dats = {'dat': dat, 'mask': mask, 'maskhm': maskhm, 'mdatname': mdatname, 'specdatname': specdatname,\
        'gfsig':gfsig, 'eshift':eshift, 'sc': test['sc'],'thresh': thresh, 'runnum':runnum, 'pixpereV': pixpereV}

In [17]:
fname = basedir.split('recon/')[1].split('/')[0]
fname = fname + '.npy'
print(fname)

torch_test_clean.npy


In [18]:
savedir = '/reg/d/psdm/tmo/tmox51020/results/paris/recon' #save in recon folder (one up)
print(os.path.join(savedir,fname))

/reg/d/psdm/tmo/tmox51020/results/paris/recon/torch_test_clean.npy


In [19]:
np.save(os.path.join(savedir,fname),dats)

In [15]:
basedir

'/reg/d/psdm/tmo/tmox51020/results/paris/recon/torch_test_clean'

In [70]:
#make sure you delete output files so you don't run out of space in the results file
! rm -r /reg/d/psdm/tmo/tmox51020/results/paris/recon/torch_test_clean/*.npy